In [1]:
import IPython.display as disp

# Quickstart Tutorial: Working with the symbolic transformed payoff matrix

The purpose of this tutorial is to show how the transformed payoff matrix can shed light on the evolutionary dynamics of a game under homophily. For this purpose, we will use a small 3-player public goods game with coordinated cooperation as our example. 

## 1. Original and transformed payoff matrices

We consider an example where the benefits returned from the public good have a sigmoid-like relationship with the number of contributors. 

Because the number of players is small ($n=3$), rather that specifying the benefits function in terms of a full sigmoid function, we instead define the benefit returned at each contribution level in terms of fixed parameters. Let $B(k)$ be the benefit returned from the public good when $k$ contribute, then we define

$$
[B(0), B(1), B(2), B(3)] = [0, \beta, b-\beta, b].
$$

We specify the coordinator quorum and contribution level $\tau = 2$. As in the general sigmoid, the cost of contributing is $c$, and the cognitive cost to Coordinating Cooperators and Liars for the capacity to understand the game and coordinate is $\varepsilon$.

We will be working with the Python library for symbolic mathematics `SymPy`, and the functions for automatically generating the payoff matrices are included in the repository.

In [2]:
import sympy as sp

import sys
sys.path.append('../functions/')

from symbolic_transformed import create_payoff_matrix
from symbolic_transformed import create_transformed_payoff_matrix
from symbolic_transformed import calc_switching_gains
from symbolic_transformed import calc_invasion_fitness_terms

The first step is to specify our payoff function, which the function `create_payoff_matrix()` will use to generate the payoff matrix $A$. The function documentation specifies that the payoff function should have the form `pi(gamma_focal, gamma_nonfocal)`. 

The input `gamma_focal` is called $\boldsymbol{\gamma}_0$ in the main text. It is a symbolic matrix of length `m` with a `1` in the `i`-th position indicating that the focal individual plays strategy `i`.

The input `gamma_nonfocal` is called $\boldsymbol{\gamma}_{-0}$ in the main text. It is a symbolic matrix of length `m` where the value of each entry `j` is how many individuals among the `n-1` nonfocal group members play strategy `j`.

The function `pi(gamma_focal, gamma_nonfocal)` will return a symbolic expression.

In [3]:
def pi(gamma_f, gamma_n):

    # fixed parameters that define the game
    # ---

    # defined
    n = 3   # group size
    tau = 2 # threshold

    # symbolic - benefit, contrib cost, cognitive cost
    b, c, eps, beta = sp.symbols('b, c, epsilon, beta')

    # benefit function, PG returns b if no. contributors >= tau
    B = lambda k: [0, beta, b-beta, b][k]

    # strategies
    strat_names = ['D', 'C', 'L', 'U']
    m = len(strat_names)   # number of strategies


    # calculate payoff to focal who plays gamma_f
    # given other members play gamma_n

    # whole group distribution
    gamma = gamma_f + gamma_n

    # count the number of each strategy in the group
    countD, countC, countL, countU = gamma

    # identify the focal strategy
    focal_strat = [strat for strat, v in zip(['D', 'C', 'L', 'U'], gamma_f) if v == 1][0]

    # payoff calculation depends on if lottery quorum is met

    if countC + countL < tau: # if the lottery quorum is not met

        k = countU                                      # only U will contribute
        benefit = B(k) # benefit from PGG

        # payoff returned depending on the strategy of the focal player
        if focal_strat == 'U':

            payoff_total = benefit - c

        elif focal_strat == 'D':

            payoff_total = benefit

        else: # focal_strat == C or L

            payoff_total = benefit - eps

    else: # lottery quorum is met

        # probability that j Coordinating Cooperators will be designated as contributors by the lottery
        denom = sp.binomial(countC + countL, tau)
        PjV = [sp.binomial(countC, j)*sp.binomial(countL, tau-j)/denom for j in range(0, tau+1)]

        # the benefit returned for each j
        benefitjV = [B(j+countU) for j in range(0, tau+1)]

        # payoff returned depending on the strategy of the focal player
        if focal_strat == 'U':

            payoff_total = sum(Pj*benefitj for Pj, benefitj in zip(PjV, benefitjV)) - c

        elif focal_strat == 'D':

            payoff_total = sum(Pj*benefitj for Pj, benefitj in zip(PjV, benefitjV))

        elif focal_strat == 'C':

            payoff_total = sum(PjV[j]*(benefitjV[j] - c*j/countC) for j in range(0, tau+1)) - eps

        else: # focal_strat == 'L':

            payoff_total = sum(Pj*benefitj for Pj, benefitj in zip(PjV, benefitjV)) - eps


    return payoff_total



The original matrix is created as follows:

In [4]:
n = 3 # number of individuals in the group
m = 4 # number of strategies

A = create_payoff_matrix(n, m, pi)
A

[[[0, 0, 0, beta], [0, b - beta, beta, beta], [0, beta, 0, beta], [beta, beta, beta, b - beta]], [[-epsilon, b - beta - c - epsilon, beta - c - epsilon, beta - epsilon], [b - beta - c - epsilon, b - beta - 2*c/3 - epsilon, b/3 + beta/3 - 2*c/3 - epsilon, b - c - epsilon], [beta - c - epsilon, b/3 + beta/3 - 2*c/3 - epsilon, 2*beta/3 - 2*c/3 - epsilon, b - beta - c - epsilon], [beta - epsilon, b - c - epsilon, b - beta - c - epsilon, b - beta - epsilon]], [[-epsilon, beta - epsilon, -epsilon, beta - epsilon], [beta - epsilon, b/3 + beta/3 - epsilon, 2*beta/3 - epsilon, b - beta - epsilon], [-epsilon, 2*beta/3 - epsilon, -epsilon, beta - epsilon], [beta - epsilon, b - beta - epsilon, beta - epsilon, b - beta - epsilon]], [[beta - c, beta - c, beta - c, b - beta - c], [beta - c, b - c, b - beta - c, b - beta - c], [beta - c, b - beta - c, beta - c, b - beta - c], [b - beta - c, b - beta - c, b - beta - c, b - c]]]

The transformed payoff matrix is created from $A$ as follows:

In [5]:
B = create_transformed_payoff_matrix(A)
B

[[[0, F_2*(b - beta)/6, 0, F_1*beta + F_2*beta/3 + F_2*(b - beta)/6], [F_2*(b - beta)/6, F_1*(b - beta) + F_2*(b - beta)/3, F_1*beta + F_2*(b - beta)/6, F_1*beta + F_2*beta/3 + F_2*(b - beta)/3], [0, F_1*beta + F_2*(b - beta)/6, 0, F_1*beta + F_2*beta/3 + F_2*(b - beta)/6], [F_1*beta + F_2*beta/3 + F_2*(b - beta)/6, F_1*beta + F_2*beta/3 + F_2*(b - beta)/3, F_1*beta + F_2*beta/3 + F_2*(b - beta)/6, F_1*(b - beta) + 2*F_2*beta/3 + F_2*(b - beta)/3]], [[-F_1*epsilon - F_2*epsilon/3 + 2*F_2*(b - beta - c - epsilon)/3 + F_3*(b - beta - 2*c/3 - epsilon), F_1*(b - beta - c - epsilon) - F_2*epsilon/6 + F_2*(b - beta - c - epsilon)/3 + F_2*(b - beta - 2*c/3 - epsilon)/2 + F_3*(b - beta - 2*c/3 - epsilon), F_1*(beta - c - epsilon) - F_2*epsilon/6 + F_2*(2*beta/3 - 2*c/3 - epsilon)/6 + F_2*(b/3 + beta/3 - 2*c/3 - epsilon)/3 + F_2*(b - beta - c - epsilon)/3 + F_3*(b - beta - 2*c/3 - epsilon), F_1*(beta - epsilon) - F_2*epsilon/6 + F_2*(b - beta - epsilon)/6 + F_2*(b - c - epsilon)/3 + F_2*(b - be

Note the subscripts used for the partition-structure probabilities $F$ refer to an indexation of the partitions of $n$, which is the ordering used in the SI about relatedness coefficients, and is the ordering in which they returned by the `utilities` function `partitionInteger()`. 

In this example, $F_1 \equiv F_{[1,1,1]}$, $F_2 \equiv F_{[2,1]}$, and $F_3 \equiv F_{[3]}$.

In [6]:
from utilities import partitionInteger

partnV = list(partitionInteger(n))
partnV

[[1, 1, 1], [1, 2], [3]]

## 2. Switching gains analysis

Taking inspiration from Peña et al. (2014), we can use the transformed payoffs to gain qualitative insights into the game under homophily. Symbolic expressions for the switching gains between pairs of strategies (along with the payoffs) can be generated using the function `calc_switching_gains()`. Below, we will do two small examples to illustrate the kind of analysis that is possible.

### 2.1 Example: Coordinating Cooperators versus Unconditional Defectors

#### No homophily

The switching gains from C (strategy 1) to D (strategy 0) under zero homophily are:

In [7]:
mix_dk, pi_foc, pi_nonfoc = calc_switching_gains(A, 0, 1)

for k, dk in enumerate(mix_dk):
    
    print(f'd_{k} =')
    disp.display(dk)

d_0 =


2*c/3 + epsilon

d_1 =


-b + beta + c + epsilon

d_2 =


epsilon

Therefore, under zero homophily, there are two possibilities for the switching-gain sign sequence.
If $-b + \beta + c + \epsilon > 0$, then the sign pattern is $(+, +, +)$, which means that D always dominates. However, we are specifically interested in the evolution of Coordinated Cooperation. Therefore, we will require a sign pattern $(+, -, +)$, which occurs when

$$
b - \beta - c - \epsilon > 0
$$

and which is a necessary condition for the coexistence of C and D. The sign pattern $(+, -, +)$ also means that Defectors can invade a population of all-Coordinating-Cooperators, but Coordinating Cooperators cannot invade a population of all-D.

#### With homophily

In [8]:
# we will simplify the expressions below by substituting in $F_1 = 1 - F_2 - F_3$
F_1, F_2, F_3 = sp.symbols('F_1, F_2, F_3')

The switching gains from C (strategy 1) to D (strategy 0) under homophily are:

In [9]:
mix_dk, pi_foc, pi_nonfoc = calc_switching_gains(B, 0, 1)

for k, dk in enumerate(mix_dk):
    
    # gather terms in expression
    dk_gather = sp.collect(sp.expand(dk), (F_1, F_2, F_3))
    print(f'd_{k} = ')
    disp.display(dk_gather)

d_0 = 


F_1*(2*c/3 + epsilon) + F_2*(-2*b/3 + 2*beta/3 + 2*c/3 + epsilon) + F_3*(-b + beta + 2*c/3 + epsilon)

d_1 = 


F_1*(-b + beta + c + epsilon) + F_2*(-2*b/3 + 2*beta/3 + 2*c/3 + epsilon) + F_3*(-b + beta + 2*c/3 + epsilon)

d_2 = 


F_1*epsilon + F_2*(-2*b/3 + 2*beta/3 + 2*c/3 + epsilon) + F_3*(-b + beta + 2*c/3 + epsilon)

Under perfect homophily, i.e., when $F_{[1,1,1]} = F_{[2,1]} = 0$ and $F_{[3]} = 1$, then all switching gains $d_0 = d_1 = d_2 = -b + \beta + \frac{2c}{3} + \epsilon < 0$, which means that Coordinating Cooperators dominate.

To analyse the effect of homophily, we simplify the expressions by substituting in $F_{[1,1,1]} = 1 - F_{[2,1]} - F_{[3]}$

In [10]:
mix_dk, pi_foc, pi_nonfoc = calc_switching_gains(B, 0, 1)

for k, dk in enumerate(mix_dk):
    
    # simplify the expression
    dk_simp = dk.subs({F_1: 1- F_2 - F_3})
    dk_simp = sp.collect(sp.expand(dk_simp), (F_2, F_3))
    
    # print it
    print(f'd_{k} =')
    disp.display(dk_simp)

d_0 =


F_2*(-2*b/3 + 2*beta/3) + F_3*(-b + beta) + 2*c/3 + epsilon

d_1 =


F_2*(b/3 - beta/3 - c/3) - F_3*c/3 - b + beta + c + epsilon

d_2 =


F_2*(-2*b/3 + 2*beta/3 + 2*c/3) + F_3*(-b + beta + 2*c/3) + epsilon

Given that $b - \beta - c - \epsilon > 0$, then $b - \beta - c > 0$, and the signs on each term are:

$$
\begin{align}
    d_0 &= 
        \underbrace{F_{[2,1]} \left(- \frac{2 b}{3} + \frac{2 \beta}{3}\right)}_{(-)}
        + \underbrace{F_{[3]} \left(- b + \beta\right)}_{(-)} 
        + \underbrace{\epsilon + \frac{2 c}{3}}_{(+)},
    \\
    d_1 &=
        \underbrace{F_{[2,1]} \left(\frac{b}{3} - \frac{\beta}{3} - \frac{c}{3}\right)}_{(+)} 
        \underbrace{- \frac{F_{[3]} c}{3}}_{(-)} 
        \underbrace{- b + \beta + c + \epsilon}_{(-)},
    \\
    d_2 &= 
        \underbrace{F_{[2,1]} \left(- \frac{2 b}{3} + \frac{2 \beta}{3} + \frac{2 c}{3}\right)}_{(-)} 
        + \underbrace{F_{[3]} \left(- b + \beta + \frac{2 c}{3}\right)}_{(-)} + \underbrace{\epsilon}_{(+)},
\end{align}
$$

We see again that, under zero homophily, i.e., $F_{[2,1]} = F_{[3]} = 0$, the switching-gain sign pattern is $(+, -, +)$. This means that D can invade all-C, C cannot invade all-D, and there is a potential for coexistence between C and D.

As homophily increases, i.e., as $F_{[2,1]}$ and $F_{[3]}$ increase from zero, $d_0$ decreases, which means that D has a harder time invading all-C. Also as homophily increases, $d_2$ decreases, which means that C has an easier time invading all-D. We know from above that, under perfect homophily, D cannot invade all-C and C can invade all-D.


### 2.2 Outputting results to LaTeX

The module `symbolic_transformed` also has a function `tabulate_switching_gains`, which will format a LaTeX table of the switching gains. 

## 3. Invasion analysis examples: Invasion of coexistence between Coordinated Cooperators and Defectors

### 3.1 Invasion of Liars into C+D

In [11]:
h1V, h2V = calc_invasion_fitness_terms(B, 1, 0, 2) # C, D, L

print('more positive means easier for L to invade')

for v in h1V:
    
    vv = v.subs({F_1: 1 - F_2 - F_3})
    vv = sp.collect(sp.expand(vv), [F_3, F_2])
    
    disp.display(vv)
    # print(sp.latex(vv))

more positive means easier for L to invade


F_2*(-2*b/3 + 2*beta/3 + 2*c/3) + F_3*(-b + beta + 2*c/3)

F_2*(2*b/9 - 8*beta/9 - c/3) + F_3*(-beta - c/3) - b + 2*beta + c

F_2*(-2*b/9 + 2*beta/9) + F_3*(-b/3 - beta/3) - 2*b/3 + 4*beta/3 + 2*c/3

Under zero homophily, Liars can invade iff $-b + 2\beta + c > 0$. If this condition is satisfied, then all $F_2$ and $F_3$ coefficients are negative. Therefore, increasing homophily impedes the invasion of Liars.

### 3.2 Invasion of Unconditional Cooperators into C+D

Sometimes the analysis is easier when the invasion-fitness terms are obtained by comparing the invader's payoffs to one or the other of the coexistence pair. Below, we comparing to Coordinating Cooperators is not as fruitful as comparing to Unconditional Defectors

In [12]:
h1V, h2V = calc_invasion_fitness_terms(B, 1, 0, 3) # C, D, U

print('more positive means easier for U to invade')

for v in h1V:
    
    vv = v.subs({F_1: 1 - F_2 - F_3})
    vv = sp.collect(sp.expand(vv), [F_3, F_2])
    
    disp.display(vv)
    # print(sp.latex(vv))

more positive means easier for U to invade


F_2*(-2*beta/3 + 2*c/3) + 2*F_3*c/3 + beta - c + epsilon

F_2*(b - 5*beta/3 - c/3) + F_3*(b - beta - c/3) - b + 2*beta + epsilon

-2*F_2*beta/3 + beta - c/3 + epsilon

In [13]:
for v in h2V:
    
    vv = v.subs({F_1: 1 - F_2 - F_3})
    vv = sp.collect(sp.expand(vv), [F_3, F_2])
    
    disp.display(vv)
    # print(sp.latex(vv))

F_2*(2*b/3 - 4*beta/3) + F_3*(b - beta) + beta - c

F_2*(2*b/3 - 4*beta/3) + F_3*(b - beta) + beta - c

F_2*(2*b/3 - 4*beta/3) + F_3*(b - beta) + beta - c

Above we see that, when homophily is zero, $\beta - c < 0$ means that U can never invade. However, because all coefficients in front of $F_2$ and $F_3$ are positive, homophily facilitates the invasion of U. Finally, because $h_2 > 0$, then U can invade C+D iff U can invade all-D.

Note that if we reversed the order of inputs of the coexisting strategies, `h1V` and `h2V` are also swapped.

In [15]:
h1V, h2V = calc_invasion_fitness_terms(B, 0, 1, 3) # D, C, U

for v in h2V:
    
    vv = v.subs({F_1: 1 - F_2 - F_3})
    vv = sp.collect(sp.expand(vv), [F_3, F_2])
    
    disp.display(vv)

F_2*(-2*beta/3 + 2*c/3) + 2*F_3*c/3 + beta - c + epsilon

F_2*(b - 5*beta/3 - c/3) + F_3*(b - beta - c/3) - b + 2*beta + epsilon

-2*F_2*beta/3 + beta - c/3 + epsilon